In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/MyDrive")

Mounted at /content/gdrive


In [ ]:
with open("twitter-training-data.txt") as f1:
    lines_train = f1.readlines()
with open("twitter-dev-data.txt") as f2:
    lines_dev = f2.readlines()

x = []
y = []
x_dev = []
y_dev = []

def load_data(lines):
    x = [] # text list
    y = [] # label list
    for value in lines:
        temp = value.split('\t')
        x.append(temp[2][:-1])
        y.append(temp[1])
    return x,y

x,y = load_data(lines_train)
x_dev,y_dev = load_data(lines_dev)


# clean the text
import re
def cleanText(textList):
    cleanTextList = []
    for text in textList:
        text = re.sub(r"@[A-Za-z0-9]+", "", text)  # remove the @mentions
        text = re.sub(r"https?:\/\/\S+", "", text)  # remove the links
        text = re.sub(r"www+", "", text)
        text = re.sub(r"#", "", text)
        cleanTextList.append(text)
    return cleanTextList
x = cleanText(x)
x_dev = cleanText(x_dev)

# clean the punctuation
def cleanPunctuation(textList): 
    punctuations = '''()-[]{};:'"\,<>./@#$%^&_~'''
    cleanTextList = []
    for val in textList:
        for x in val.lower(): 
            if x in punctuations: 
                val = val.replace(x, " ") 
        cleanTextList.append(val)
    return cleanTextList
x = cleanPunctuation(x)
x_dev = cleanPunctuation(x_dev)


from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=None, split=' ')
tokenizer.fit_on_texts(x + x_dev)

from keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(x)
X = pad_sequences(X,maxlen = 50)

X_dev = tokenizer.texts_to_sequences(x_dev)
X_dev = pad_sequences(X_dev,maxlen = 50)

import numpy as np

# use onehot
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
def Label_OneHot(list):
    label_encoder = LabelEncoder()
    label = label_encoder.fit_transform(list)
    onehot_encoder = OneHotEncoder(sparse=False)
    label = label.reshape(len(label), 1)
    label = onehot_encoder.fit_transform(label)
    return label
Y = Label_OneHot(y)
Y_dev = Label_OneHot(y_dev)
print("X_train")
print(X.shape)
print(X_dev.shape)
print("Y_train")
print(Y)


X_train
(45101, 50)
(2000, 50)
Y_train
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]]


In [ ]:

import numpy as np
# 读glove数据的函数
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() 
        word2vector = {}
        for line in f:
            line_ = line.strip() 
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = np.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector



w_idx = tokenizer.word_index
print(w_idx)
vocab, word_to_idx = read_data("glove.6B.100d.txt")
embedding_matrix = np.zeros((len(w_idx) + 1, 100))
for word, i in w_idx.items():
    embedding_vector = word_to_idx.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


{'the': 1, 'to': 2, 'i': 3, 'a': 4, 'in': 5, 'and': 6, 'on': 7, 'of': 8, 's': 9, 'is': 10, 'for': 11, 'it': 12, 'you': 13, 'with': 14, 'be': 15, 'tomorrow': 16, 'at': 17, 'may': 18, 'that': 19, 't': 20, 'my': 21, 'this': 22, 'have': 23, 'he': 24, 'but': 25, 'just': 26, 'was': 27, 'day': 28, 'will': 29, 'not': 30, 'so': 31, 'all': 32, 'we': 33, 'are': 34, 'm': 35, 'me': 36, 'out': 37, 'if': 38, 'can': 39, 'going': 40, 'from': 41, 'night': 42, 'see': 43, 'about': 44, 'as': 45, 'like': 46, 'up': 47, '1st': 48, 'sunday': 49, 'friday': 50, 'time': 51, 'get': 52, 'by': 53, 'his': 54, 'one': 55, 'what': 56, 'they': 57, 'go': 58, '2': 59, 'new': 60, 'now': 61, 'has': 62, 'or': 63, 'when': 64, 'your': 65, 'saturday': 66, 'amp': 67, '2nd': 68, 'no': 69, 'monday': 70, 'do': 71, 'an': 72, 'game': 73, 'today': 74, 'who': 75, 'good': 76, 'don': 77, 'know': 78, '3rd': 79, 'want': 80, '3': 81, 'she': 82, 'there': 83, 'tonight': 84, 'll': 85, '1': 86, 'how': 87, 'think': 88, 'make': 89, 'after': 90, 'l

In [ ]:
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
model_lstm_gwe = Sequential()
model_lstm_gwe.add(Embedding(len(w_idx) +1 ,100,input_length = X.shape[1], weights=[embedding_matrix],trainable=False))
model_lstm_gwe.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
#model_lstm_gwe.add(Dense(64, activation='relu'))
model_lstm_gwe.add(keras.layers.core.Dense(3, activation='softmax'))
#adam rmsprop 
model_lstm_gwe.compile(loss = "categorical_crossentropy", optimizer='adam',metrics = ['accuracy'])
print(model_lstm_gwe.summary())

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 50, 100)           4609700   
_________________________________________________________________
lstm_20 (LSTM)               (None, 300)               481200    
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 903       
Total params: 5,091,803
Trainable params: 482,103
Non-trainable params: 4,609,700
_________________________________________________________________
None


In [ ]:
import tensorflow as tf
#tf.compat.v1.disable_eager_execution()
model_lstm_gwe.fit(X, Y, epochs = 10, batch_size=32)


Train on 45101 samples
Epoch 1/10
45101/45101 [==============================] - 161s 4ms/sample - loss: 0.8587 - accuracy: 0.5850
Epoch 2/10
45101/45101 [==============================] - 152s 3ms/sample - loss: 0.7865 - accuracy: 0.6312
Epoch 3/10
45101/45101 [==============================] - 147s 3ms/sample - loss: 0.7539 - accuracy: 0.6487
Epoch 4/10
45101/45101 [==============================] - 147s 3ms/sample - loss: 0.7228 - accuracy: 0.6677
Epoch 5/10
45101/45101 [==============================] - 146s 3ms/sample - loss: 0.6881 - accuracy: 0.6847
Epoch 6/10
45101/45101 [==============================] - 145s 3ms/sample - loss: 0.6546 - accuracy: 0.7018
Epoch 7/10
45101/45101 [==============================] - 145s 3ms/sample - loss: 0.6135 - accuracy: 0.7230
Epoch 8/10
45101/45101 [==============================] - 144s 3ms/sample - loss: 0.5686 - accuracy: 0.7481
Epoch 9/10
45101/45101 [==============================] - 144s 3ms/sample - loss: 0.5310 - accuracy: 0.7688
Epoch

In [ ]:
print(model_lstm_gwe.evaluate(X_dev, Y_dev)[1])

0.6555
